In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-jun-2022/data.csv


In [2]:
import torch 
from tqdm.notebook import tqdm as tqdm
import torch.nn.functional as func 

In [3]:
df = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv')
res = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv')

In [4]:
df.drop('row_id',axis = 1,inplace = True)
df.head()

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,...,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,...,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943


In [5]:
df.fillna(0, inplace = True)
df.head()

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,0.000000,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,...,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,...,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 80 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   F_1_0   1000000 non-null  float64
 1   F_1_1   1000000 non-null  float64
 2   F_1_2   1000000 non-null  float64
 3   F_1_3   1000000 non-null  float64
 4   F_1_4   1000000 non-null  float64
 5   F_1_5   1000000 non-null  float64
 6   F_1_6   1000000 non-null  float64
 7   F_1_7   1000000 non-null  float64
 8   F_1_8   1000000 non-null  float64
 9   F_1_9   1000000 non-null  float64
 10  F_1_10  1000000 non-null  float64
 11  F_1_11  1000000 non-null  float64
 12  F_1_12  1000000 non-null  float64
 13  F_1_13  1000000 non-null  float64
 14  F_1_14  1000000 non-null  float64
 15  F_2_0   1000000 non-null  int64  
 16  F_2_1   1000000 non-null  int64  
 17  F_2_2   1000000 non-null  int64  
 18  F_2_3   1000000 non-null  int64  
 19  F_2_4   1000000 non-null  int64  
 20  F_2_5   1000000 non-null 

In [7]:
data = df.to_numpy()
data

array([[-3.545910e-01, -4.640380e-01,  2.304115e+00, ...,  3.809505e+00,
         1.236486e+00,  1.182055e+00],
       [ 1.380940e+00, -4.996260e-01, -4.185480e-01, ..., -4.218259e+00,
        -2.724883e+00, -6.377500e-02],
       [ 2.560230e-01, -1.059874e+00,  0.000000e+00, ..., -2.131747e+00,
         3.661499e+00, -1.315760e-01],
       ...,
       [ 1.475340e-01, -7.152760e-01, -4.650490e-01, ..., -5.841080e-01,
        -1.492096e+00, -9.975020e-01],
       [-1.709886e+00, -8.137850e-01, -1.866536e+00, ..., -1.085554e+00,
         3.122423e+00,  4.831000e-03],
       [-8.063800e-01, -2.525100e-02, -8.754770e-01, ...,  2.334131e+00,
         5.425421e+00, -8.288470e-01]])

In [40]:
# System Parameters;
min_batch = 128
miss_rate = 0.018
train_rate = 0.8
alpha = 10
# 3. Hint rate
p_hint = 0.9

num = data.shape[0]
dim = data.shape[1]

h_dim1 = dim
h_dim2 = dim

min_val = np.zeros(dim)
max_val = np.zeros(dim)
rng = np.random.RandomState()
#Normalization
for i in range(dim):
    min_val[i] = np.min(data[:,i])
    data[:,i] = data[:,i] - np.min(data[:,i])
    max_val[i] = np.max(data[:,i])
    data[:,i] = data[:,i]/(np.max(data[:,i])+ 1e-6)
#Missing value matrix   
mat_miss_ind = int(num*miss_rate)
missing_mat = np.ones(data.shape)
for i in range(dim):
    A = missing_mat[:,i]
    A[:mat_miss_ind] = 0.
    rng.shuffle(A)
    missing_mat[:,i] = A

rand_index = np.random.permutation(num)
train_ind = int(num * train_rate)

trainX = data[rand_index[:train_ind], :]
testX = data[rand_index[train_ind:], :]

MisstrainX = missing_mat[rand_index[:train_ind], :]
MisstestX = missing_mat[rand_index[train_ind:], :]

In [9]:
def sample_X (new_x, size):
    ind = np.random.permutation(new_x)
    return ind[:size]

In [42]:
def sample_Z (m,n):
    return np.random.uniform(0.,0.01,size=[m,n])

def sample_M (u,v,rate):
    A = np.ones((u,v))
    miss_ind = int(u*rate)
    for i in range(v):
        B = A[:,i]
        B[:miss_ind] = 0.
        rng.shuffle(B)
        A[:,i] = B
    return A
        
def xavier_init(size):
    stddev = 1/np.sqrt(size[0]/2.)
    return np.random.normal(size=size,scale =stddev)


In [13]:
#Generator architecture
G_w1 = torch.tensor(xavier_init([dim*2,h_dim1]),requires_grad = True)     # Data + Mask
G_b1 = torch.tensor(np.zeros(h_dim1), requires_grad = True)

G_w2 = torch.tensor(xavier_init([h_dim1,h_dim2]),requires_grad = True)
G_b2 = torch.tensor(np.zeros(h_dim2), requires_grad = True)

G_w3 = torch.tensor(xavier_init([h_dim2,dim]),requires_grad = True)
G_b3 = torch.tensor(np.zeros(dim), requires_grad = True)

delta_G = [G_w1,G_w2,G_w3,G_b1,G_b2,G_b3]

# Discrimnator architecture
D_w1 = torch.tensor(xavier_init([dim*2,h_dim1]),requires_grad = True)      # Data + Hint
D_b1 = torch.tensor(np.zeros(h_dim1), requires_grad = True)

D_w2 = torch.tensor(xavier_init([h_dim1, h_dim2]),requires_grad = True)
D_b2 = torch.tensor(np.zeros(h_dim2), requires_grad = True)

D_w3 = torch.tensor(xavier_init([h_dim2,dim]),requires_grad = True)
D_b3 = torch.tensor(np.zeros(dim), requires_grad = True)

delta_D = [D_w1,D_w2,D_w3,D_b1,D_b2,D_b3]

In [14]:
def generator(new_x,mask):
    inp = torch.cat((new_x,mask),1)
    G_h1 = func.relu(torch.matmul(inp,G_w1) + G_b1)
    G_h2 = func.relu(torch.matmul(G_h1,G_w2) + G_b2)
    G_output = torch.sigmoid(torch.matmul(G_h2,G_w3)+G_b3)
    return G_output 

def discrimnator(new_x,hint):
    inp = torch.cat((new_x,hint),1)
    D_h1 = func.relu(torch.matmul(inp,D_w1)+D_b1)
    D_h2 = func.relu(torch.matmul(D_h1,D_w2)+D_b2)
    D_output = torch.sigmoid(torch.matmul(D_h2,D_w3)+D_b3)
    return D_output


In [15]:
def discrimnator_loss(mask,hint,new_x):
    gen = generator(new_x,mask)
    h_new_x = mask*new_x + (1-mask)*gen
    disc = discrimnator(h_new_x,hint)
    LD = -torch.mean(mask*torch.log(disc + 1e-8) + (1-mask)*torch.log(1-disc + 1e-8))
    return LD 

def generator_loss(x,new_x,mask,hint):
    gen = generator(new_x,mask)
    h_new_x = mask*new_x + (1-mask)*gen
    
    disc = discrimnator(h_new_x,hint)
    LG = -torch.mean((1-mask)*torch.log(disc + 1e-8))
    
    Lm_ini = mask*new_x
    Lm_out = mask*gen
    
    LM = torch.mean((Lm_ini - Lm_out)**2)/torch.mean(mask)
    
    gen_loss = LG + alpha*LM
    mse_test_loss = torch.mean(((1-mask)*x - (1-mask)*gen)**2)/torch.mean(1-mask)
    
    return gen_loss, LM, mse_test_loss

In [16]:
def test_loss(x,new_x,m):
    gen = generator(new_x,m)
    mse_test_loss = torch.mean(((1-m)*x - (1-m)*gen)**2)/torch.mean(1-m)
    
    return gen, mse_test_loss   

In [19]:
optimD = torch.optim.Adam(params = delta_D)
optimG = torch.optim.Adam(params = delta_G)

In [45]:
for it in tqdm(range(50000)):
    indX = sample_X(train_ind, min_batch)
    X = trainX[indX,:]
    Z = sample_Z(min_batch,dim)
    M = MisstrainX[indX,:]
    H = M * sample_M(min_batch,dim,1-miss_rate)
    new_x = M * X + (1-M)*Z
    
    new_x = torch.tensor(new_x)
    X = torch.tensor(X)
    M = torch.tensor(M)
    H = torch.tensor(H)
    
    optimD.zero_grad(set_to_none=False)
    currD_loss = discrimnator_loss(M,H,new_x)
    currD_loss.backward()
    optimD.step()
    
    optimG.zero_grad(set_to_none=False)
    currG_to_loss,mse_train_loss,mse_test_loss = generator_loss(X,new_x,M,H)
    currG_to_loss.backward()
    optimG.step()
    
    if it%1000 == 0:
        print("For Iter: {n:}".format(n=it),end = '/t')
        print("Train loss : {test:.4} and Test loss: {train:.4}".format(test=np.sqrt(mse_train_loss.item()),train=np.sqrt(mse_test_loss.item())))
        

  0%|          | 0/50000 [00:00<?, ?it/s]

For Iter: 0/tTrain loss : 0.3162 and Test loss: 0.3322
For Iter: 1000/tTrain loss : 0.09408 and Test loss: 0.1086
For Iter: 2000/tTrain loss : 0.08661 and Test loss: 0.114
For Iter: 3000/tTrain loss : 0.0795 and Test loss: 0.1177
For Iter: 4000/tTrain loss : 0.07769 and Test loss: 0.1093
For Iter: 5000/tTrain loss : 0.07631 and Test loss: 0.109
For Iter: 6000/tTrain loss : 0.07678 and Test loss: 0.1168
For Iter: 7000/tTrain loss : 0.07594 and Test loss: 0.09787
For Iter: 8000/tTrain loss : 0.07658 and Test loss: 0.09332
For Iter: 9000/tTrain loss : 0.07501 and Test loss: 0.1133
For Iter: 10000/tTrain loss : 0.07435 and Test loss: 0.1019
For Iter: 11000/tTrain loss : 0.07466 and Test loss: 0.1028
For Iter: 12000/tTrain loss : 0.07527 and Test loss: 0.1026
For Iter: 13000/tTrain loss : 0.07562 and Test loss: 0.1071
For Iter: 14000/tTrain loss : 0.0744 and Test loss: 0.09849
For Iter: 15000/tTrain loss : 0.07479 and Test loss: 0.1067
For Iter: 16000/tTrain loss : 0.07551 and Test loss: 0.

In [46]:
test_x = testX
test_no = num - train_ind
Z = sample_Z(test_no,dim)
M = missing_mat[rand_index[train_ind:], :]
new_test_x = M*test_x + (1-M)*Z

M = torch.tensor(M)
new_test_x = torch.tensor(new_test_x)

gen, mse_test_loss = test_loss(test_x,new_test_x,M)

print("Test Loss: {val:.4}".format(val=np.sqrt(mse_test_loss.item())))


Test Loss: 0.1025


In [47]:
imputed_data = (M * test_x + (1-M) * gen).detach().numpy()

X_train = torch.tensor(trainX)
M_train = torch.tensor(MisstrainX)
sample = currG_to_loss

imputed_data_train = (M_train * X_train + (1-M_train) * sample).detach().numpy()

In [48]:
final_set_train = np.zeros(trainX.shape)
for i in range(dim):
    minv = min_val[i]
    maxv = max_val[i]
    final_set_train[:,i] = imputed_data_train[:,i]*maxv + minv
    

final_set_test = np.zeros(testX.shape)
for i in range(dim):
    minv = min_val[i]
    maxv = max_val[i]
    final_set_test[:,i] = imputed_data[:,i]*maxv + minv

In [49]:
result = df.to_numpy()
result[rand_index[:train_ind]]

array([[-0.618753,  0.221454,  1.15471 , ...,  4.569999, -1.75299 ,
         0.096553],
       [-0.105178,  0.264646,  0.413251, ...,  1.670806, -1.026491,
        -1.745219],
       [ 1.059536,  0.575812,  1.075676, ...,  0.      ,  1.86339 ,
         0.270463],
       ...,
       [ 1.128586, -2.067664, -0.016712, ...,  4.895038, -0.070414,
         0.74611 ],
       [-0.316467, -0.397591, -0.51135 , ...,  1.515648,  1.168376,
         0.881015],
       [ 0.753785, -0.646093, -0.345656, ...,  2.326378,  4.297027,
         0.479601]])

In [50]:
final_set_train

array([[0.41692525, 0.50969527, 0.60249393, ..., 0.70384236, 0.39484188,
        0.80204381],
       [0.46985125, 0.51408764, 0.52835314, ..., 0.5770524 , 0.42703995,
        0.65550224],
       [0.58987978, 0.54573136, 0.59459107, ..., 0.5039833 , 0.55511803,
        0.81588105],
       ...,
       [0.59699566, 0.27690568, 0.48535981, ..., 0.71805724, 0.46941281,
        0.85372616],
       [0.44807706, 0.4467421 , 0.43589942, ..., 0.5702669 , 0.52431536,
        0.86445994],
       [0.55837089, 0.42147093, 0.45246768, ..., 0.60572243, 0.6629756 ,
        0.83252123]])

In [51]:
result[rand_index[:train_ind]] = final_set_train
result[rand_index[:train_ind]]

array([[0.41692525, 0.50969527, 0.60249393, ..., 0.70384236, 0.39484188,
        0.80204381],
       [0.46985125, 0.51408764, 0.52835314, ..., 0.5770524 , 0.42703995,
        0.65550224],
       [0.58987978, 0.54573136, 0.59459107, ..., 0.5039833 , 0.55511803,
        0.81588105],
       ...,
       [0.59699566, 0.27690568, 0.48535981, ..., 0.71805724, 0.46941281,
        0.85372616],
       [0.44807706, 0.4467421 , 0.43589942, ..., 0.5702669 , 0.52431536,
        0.86445994],
       [0.55837089, 0.42147093, 0.45246768, ..., 0.60572243, 0.6629756 ,
        0.83252123]])

In [52]:
final_set_test

array([[0.44938605, 0.36476269, 0.59349655, ..., 0.62889244, 0.4272311 ,
        0.70614244],
       [0.4957607 , 0.6081064 , 0.48738067, ..., 0.63116279, 0.51879785,
        0.74192775],
       [0.30266632, 0.62778236, 0.53611001, ..., 0.62372987, 0.57894344,
        0.79969392],
       ...,
       [0.52385921, 0.44516066, 0.42266034, ..., 0.51204904, 0.60660253,
        0.74539999],
       [0.54175507, 0.35085685, 0.52947117, ..., 0.66513809, 0.56487031,
        0.80746524],
       [0.36632993, 0.39450992, 0.42011731, ..., 0.55749389, 0.39936677,
        0.80710091]])

In [53]:
result[rand_index[train_ind:]] = final_set_test
result[rand_index[train_ind:]]

array([[0.44938605, 0.36476269, 0.59349655, ..., 0.62889244, 0.4272311 ,
        0.70614244],
       [0.4957607 , 0.6081064 , 0.48738067, ..., 0.63116279, 0.51879785,
        0.74192775],
       [0.30266632, 0.62778236, 0.53611001, ..., 0.62372987, 0.57894344,
        0.79969392],
       ...,
       [0.52385921, 0.44516066, 0.42266034, ..., 0.51204904, 0.60660253,
        0.74539999],
       [0.54175507, 0.35085685, 0.52947117, ..., 0.66513809, 0.56487031,
        0.80746524],
       [0.36632993, 0.39450992, 0.42011731, ..., 0.55749389, 0.39936677,
        0.80710091]])

In [54]:
result

array([[0.44414823, 0.43998484, 0.7174265 , ..., 0.6705838 , 0.52733396,
        0.88841235],
       [0.62300177, 0.43636575, 0.44517898, ..., 0.31950683, 0.35176807,
        0.78928722],
       [0.50707449, 0.37939188, 0.48703089, ..., 0.41075594, 0.63480932,
        0.7838926 ],
       ...,
       [0.49589425, 0.47559618, 0.4405292 , ..., 0.48784436, 0.40640457,
        0.71499474],
       [0.30447955, 0.40441766, 0.30039017, ..., 0.45650894, 0.61091774,
        0.7947459 ],
       [0.39758953, 0.48460685, 0.39948924, ..., 0.60606149, 0.71298546,
        0.72841386]])

In [56]:
result = result.reshape(80,1000000)

In [57]:
ans = pd.DataFrame(result,index=list(df.columns))
ans.head(6)

,0,1,2,3,4,5,6,7,8,9,...,999990,999991,999992,999993,999994,999995,999996,999997,999998,999999
F_1_0,0.444148,0.439985,0.717426,0.550378,0.690761,0.539166,0.460840,0.787248,0.287124,0.459515,...,0.452837,0.497565,0.596678,0.802597,0.055116,0.789514,0.493720,0.454989,0.435071,0.823387
F_1_1,0.487940,0.387962,0.342734,0.433148,0.567018,0.573500,0.588145,0.770924,0.480413,0.460437,...,0.507039,0.625399,0.493115,0.769459,0.781098,0.777231,0.480120,0.489384,0.418727,0.795440
F_1_2,0.435798,0.293478,0.617359,0.340341,0.529532,0.302307,0.476647,0.757984,0.516173,0.569385,...,0.419238,0.368480,0.477585,0.834987,0.764171,0.767288,0.515502,0.607371,0.472534,0.916405
F_1_3,0.446829,0.487175,0.476223,0.480530,0.415416,0.494383,0.623866,0.636480,0.389068,0.588349,...,0.546074,0.592219,0.665611,0.803964,0.789341,0.809394,0.458642,0.568656,0.420154,0.748403
F_1_4,0.475259,0.294165,0.413495,0.545859,0.528448,0.515489,0.369063,0.564205,0.516917,0.413280,...,0.525413,0.379308,0.559791,0.755506,0.795274,0.757794,0.419973,0.646004,0.626972,0.795972
F_1_5,0.537493,0.547273,0.433632,0.461257,0.472466,0.516880,0.628524,0.688602,0.505167,0.488864,...,0.616965,0.519109,0.570031,0.857274,0.739132,0.823362,0.404749,0.547090,0.351243,0.645636


In [58]:
for i,col,val in zip(range(res.shape[0]),res['row-col'],res['value']):
    x = col.split("-")
    val = ans.loc[x[1],int(x[0])]
    res['value'][i] = val

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [59]:
res

,row-col,value
0,0-F_1_14,0.266342
1,0-F_3_23,0.594269
2,1-F_3_24,0.603969
3,2-F_1_2,0.617359
4,2-F_4_2,0.300113
...,...,...
999995,999993-F_4_2,0.887132
999996,999994-F_3_10,0.674869
999997,999994-F_4_9,0.778133
999998,999997-F_3_14,0.313218


In [60]:
res.to_csv('Output_Playgroundjune2022',index= False)